In [2]:
import pandas as pd
from datetime import datetime, timedelta, time
import numpy as np
from scipy.ndimage.interpolation import shift
from collections import defaultdict
import seaborn as sns

import calendar

In [3]:
df = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_181229.txt')

In [4]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/22/2018,03:00:00,REGULAR,6881995,2333630
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/22/2018,07:00:00,REGULAR,6882006,2333643
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/22/2018,11:00:00,REGULAR,6882062,2333740
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/22/2018,15:00:00,REGULAR,6882243,2333800
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/22/2018,19:00:00,REGULAR,6882590,2333868


In [5]:
#Check data types 
df.dtypes

C/A                                                                     object
UNIT                                                                    object
SCP                                                                     object
STATION                                                                 object
LINENAME                                                                object
DIVISION                                                                object
DATE                                                                    object
TIME                                                                    object
DESC                                                                    object
ENTRIES                                                                  int64
EXITS                                                                    int64
dtype: object

In [6]:
#Convert DATES to a datetime object
df['DATE'] =  pd.to_datetime(df['DATE'])

In [8]:
#Create column for day reference
df['DAY'] = df['DATE'].dt.weekday_name

In [14]:
df['MONTH']=df['DATE'].dt.month

In [20]:
df.tail()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DAY,MONTH,DELTA
201169,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2018-12-28,11:12:57,REGULAR,5554,366,Friday,12,0.0
201170,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2018-12-28,12:00:00,REGULAR,5554,366,Friday,12,0.0
201171,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2018-12-28,12:19:32,REGULAR,5554,366,Friday,12,0.0
201172,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2018-12-28,16:00:00,REGULAR,5554,366,Friday,12,0.0
201173,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2018-12-28,20:00:00,REGULAR,5554,366,Friday,12,0.0


In [38]:
df['DELTA'] = df['ENTRIES'].diff().abs()

In [39]:
#Clean columns "EXITS" cant referenced because of space
df = df.rename(columns=lambda x: x.strip())

In [41]:
df['DELTA'] = df['ENTRIES'].diff().abs()

In [42]:
df['DELTA2'] = df['EXITS'].diff().abs()

In [42]:
df['TRAFFIC']

In [35]:
df_station=df.groupby(['STATION','DAY','MONTH'])['DELTA'].sum().reset_index()

In [37]:
df_station


,STATION,DAY,MONTH,DELTA
0,1 AV,Friday,12,1.422700e+04
1,1 AV,Monday,12,1.036200e+04
2,1 AV,Saturday,12,2.377804e+09
3,1 AV,Sunday,12,9.707000e+03
4,1 AV,Thursday,12,1.376400e+04
5,1 AV,Tuesday,12,5.509000e+03
6,1 AV,Wednesday,12,1.183700e+04
7,103 ST,Friday,12,2.333300e+04
8,103 ST,Monday,12,1.923600e+04
9,103 ST,Saturday,12,7.248141e+08


In [24]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DAY,MONTH,DELTA
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-12-22,03:00:00,REGULAR,6881995,2333630,Saturday,12,NaN
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-12-22,07:00:00,REGULAR,6882006,2333643,Saturday,12,11.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-12-22,11:00:00,REGULAR,6882062,2333740,Saturday,12,56.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-12-22,15:00:00,REGULAR,6882243,2333800,Saturday,12,181.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-12-22,19:00:00,REGULAR,6882590,2333868,Saturday,12,347.0
